<a href="https://colab.research.google.com/github/abazabaaa/colab_tutorial/blob/main/equibind_testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
import sys
!pip -q install py3Dmol 2>&1 1>/dev/null
!pip install Cython

# install conda
!wget -qnc https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh 
!bash Miniconda3-latest-Linux-x86_64.sh -bfp /usr/local 2>&1 1>/dev/null

In [2]:
!rm -r Miniconda3-latest-Linux-x86_64.sh
!conda install -y -q -c conda-forge python=3.7 pdbfixer 2>&1 1>/dev/null
!conda install -c conda-forge parmed  --yes 2>&1 1>/dev/null
!conda install -c bioconda pybel --yes
!conda install -c openbabel openbabel --yes

Solving environment: | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / failed with initial frozen solve. Retrying with flexible solve.
Solving environment: \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - failed with repodata from current_repodata.json, will retry with next repodata source.
Solving environment: \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / 

In [4]:
!git clone https://github.com/rlabduke/reduce.git
!sudo make -C /content/reduce install


Cloning into 'reduce'...
remote: Enumerating objects: 2915, done.
remote: Counting objects: 100% (377/377), done.
remote: Compressing objects: 100% (130/130), done.
remote: Total 2915 (delta 248), reused 356 (delta 246), pack-reused 2538
Receiving objects: 100% (2915/2915), 27.23 MiB | 12.37 MiB/s, done.
Resolving deltas: 100% (2093/2093), done.


In [17]:
!pip install torch
!pip install torchvision
!pip install dgl-cu102
!pip install bio
!pip install biopandas
!pip install joblib
!pip install tensorboard
!pip install POT
!pip install pyaml
!pip install rdkit-pypi

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 776.3/776.3 MB 1.8 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.1/19.1 MB 42.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 59.9 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.8/36.8 MB 29.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.1/38.1 MB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 40.5 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [18]:
#load dependencies
sys.path.append('/usr/local/lib/python3.7/site-packages/')
import parmed as pmd
from biopandas.pdb import PandasPdb
import os
import urllib.request  
import numpy as np
import py3Dmol
from __future__ import print_function
import platform
import scipy.cluster.hierarchy
from scipy.spatial.distance import squareform
import scipy.stats as stats
import matplotlib.pyplot as plt
import pandas as pd
from scipy.interpolate import griddata
import seaborn as sb
from statistics import mean, stdev
from matplotlib import colors
from IPython.display import set_matplotlib_formats

In [19]:
#@title **Check if you correctly allocated GPU nodes**

gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, 
and then re-execute this cell.


In [20]:
#@title **Please, provide the necessary input files below for receptor**: 
#@markdown **Important:** Run the cell to prepare your receptor.

import warnings
warnings.filterwarnings('ignore')
import os
from Bio.PDB import PDBParser, PDBIO, Select
from Bio.PDB import is_aa
import pandas as pd
from pdbfixer import PDBFixer


# Google_Drive_Path = '/content/' #@param {type:"string"}
workDir = '/content/'

if os.path.exists(os.path.join(workDir, "name_residues_receptor.txt")):
  os.remove(os.path.join(workDir,"name_residues_receptor.txt"))
else:
  pass

receptor = os.path.join(workDir, "receptor.pdb")
ligand = os.path.join(workDir, "ligand.sdf")


Query_PDB_ID = '3HTB' #@param {type:"string"}

pdbfn = Query_PDB_ID + ".pdb"
url = 'https://files.rcsb.org/download/' + pdbfn
outfnm = os.path.join(workDir, pdbfn)
urllib.request.urlretrieve(url, outfnm)

#prepare receptor
ppdb = PandasPdb().read_pdb(outfnm)
ppdb.df['ATOM'] = ppdb.df['ATOM']
ppdb.df['HETATM'] = ppdb.df['HETATM'][ppdb.df['HETATM']['residue_name'] != 'HOH']
ppdb.df['ATOM'] = ppdb.df['ATOM'][ppdb.df['ATOM']['atom_name'] != 'OXT']
ppdb.to_pdb(path=receptor, records=['ATOM', 'HETATM'], gz=False, append_newline=True)

path = '/content/'

def is_het(residue):
    res = residue.id[0]
    return res != " " and res != "W"

def aa(residue):
    res = residue.id[0]
    return res != "W" 


class ResidueSelect(Select):
    def __init__(self, chain, residue):
        self.chain = chain
        self.residue = residue

    def accept_chain(self, chain):
        return chain.id == self.chain.id

    def accept_residue(self, residue):
        return residue == self.residue and aa(residue)

def extract_ligands(path):
    pdb = PDBParser().get_structure(receptor, receptor)
    io = PDBIO()
    io.set_structure(pdb)
    i2 = 1
    name_residues2 = []
    for model in pdb:
      for chain in model:
        for residue in chain:
          if not aa(residue):
            continue
          # print(f"{chain[i].resname} {i}")
          name_residues2.append(residue)
          print((f"saving {residue}"), file=open(os.path.join(workDir, "name_residues_receptor.txt"), "a",))
          i2 += 1

extract_ligands(path)


dataset2 = pd.read_csv(os.path.join(workDir, 'name_residues_receptor.txt'), delimiter = " ", header=None)
df2 = pd.DataFrame(dataset2)
df2 = df2.iloc[:, [2]]
new2 = df2.to_numpy()

if os.path.exists(receptor):
  print("Successfully generated the files! :-)")
else:
  print("ERROR: Check your inputs! ")

Successfully generated the files! :-)


In [25]:
!reduce -FLIP receptor.pdb >> receptor2.pdb

Processing file: "receptor.pdb"
Building His ring NH Hydrogens.
Building or keeping OH & SH Hydrogens.
Rotating existing OH & SH Hydrogens
        if warning inappropriate, adjust the cuttoffs using -METALBump or -NONMETALBump.
        if warning inappropriate, adjust the cuttoffs using -METALBump or -NONMETALBump.
        if warning inappropriate, adjust the cuttoffs using -METALBump or -NONMETALBump.
        if warning inappropriate, adjust the cuttoffs using -METALBump or -NONMETALBump.
        if warning inappropriate, adjust the cuttoffs using -METALBump or -NONMETALBump.
        if warning inappropriate, adjust the cuttoffs using -METALBump or -NONMETALBump.
        if warning inappropriate, adjust the cuttoffs using -METALBump or -NONMETALBump.
        if warning inappropriate, adjust the cuttoffs using -METALBump or -NONMETALBump.
        if warning inappropriate, adjust the cuttoffs using -METALBump or -NONMETALBump.
        if warning inappropriate, adjust the cuttoffs using 

/usr/bin/make


make: Entering directory '/content/reduce'
installing /toolclasses...
make[1]: Entering directory '/content/reduce/toolclasses'
g++ -O  -DCHARFUNCMACROS -DINCTEMPLATEDEFNS -DBOOLPREDEFINED -DBRACKETOPERPARMS -DLISTFRIENDFIX -c UseCount.cpp -o UseCount.o
g++ -O  -DCHARFUNCMACROS -DINCTEMPLATEDEFNS -DBOOLPREDEFINED -DBRACKETOPERPARMS -DLISTFRIENDFIX -c Point3d.cpp -o Point3d.o
g++ -O  -DCHARFUNCMACROS -DINCTEMPLATEDEFNS -DBOOLPREDEFINED -DBRACKETOPERPARMS -DLISTFRIENDFIX -c utility.cpp -o utility.o
g++ -O  -DCHARFUNCMACROS -DINCTEMPLATEDEFNS -DBOOLPREDEFINED -DBRACKETOPERPARMS -DLISTFRIENDFIX -c column_sprintf.cpp -o column_sprintf.o
g++ -O  -DCHARFUNCMACROS -DINCTEMPLATEDEFNS -DBOOLPREDEFINED -DBRACKETOPERPARMS -DLISTFRIENDFIX -c column_sscanf.cpp -o column_sscanf.o
g++ -O  -DCHARFUNCMACROS -DINCTEMPLATEDEFNS -DBOOLPREDEFINED -DBRACKETOPERPARMS -DLISTFRIENDFIX -c DisjointSets.cpp -o DisjointSets.o
Loading libtoolclasses.a ... 
ar: `u' modifier ignored since `D' is the default (see `U')


/usr/local/bin/reduce


reduce	sample_data
